# Page Rank
PageRank is named after Google co-founder Larry Page and is used to rank websites in Google's search results.
It counts the number and quality of links to a page to determine an estimate of how important a page is.
The underlying assumption is that more important pages are likely to receive more links from other pages.

First we'll import the Neo4j driver and Pandas libraries:


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

Next let's create an instance of the Neo4j driver which we'll use to execute our queries.


In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

Now let's create a sample graph that we'll run the algorithm against.


In [3]:
create_graph_query = '''
CREATE (home:Page{name:'Home'})
,(about:Page{name:'About'})
,(product:Page{name:'Product'})
,(links:Page{name:'Links'})
,(a:Page{name:'Site A'})
,(b:Page{name:'Site B'})
,(c:Page{name:'Site C'})
,(d:Page{name:'Site D'})
CREATE (home)-[:LINKS]->(about)
,(about)-[:LINKS]->(home)
,(product)-[:LINKS]->(home)
,(home)-[:LINKS]->(product)
,(links)-[:LINKS]->(home)
,(home)-[:LINKS]->(links)
,(links)-[:LINKS]->(a)-[:LINKS]->(home)
,(links)-[:LINKS]->(b)-[:LINKS]->(home)
,(links)-[:LINKS]->(c)-[:LINKS]->(home)
,(links)-[:LINKS]->(d)-[:LINKS]->(home)
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 8, 'relationships-created': 14, 'nodes-created': 8, 'properties-set': 8}


Finally we can run the algorithm by executing the following query:


In [4]:
streaming_query = """
CALL algo.pageRank.stream('Page', 'LINKS', {iterations:20, dampingFactor:0.85})
YIELD node, score
RETURN node.name AS page,score
ORDER BY score DESC
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,page,score
0,Home,3.233877
1,Links,1.060435
2,About,1.060435
3,Product,1.060435
4,Site A,0.329035
5,Site B,0.329035
6,Site C,0.329035
7,Site D,0.329035
